In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import html
import json
import time
from requests.exceptions import ConnectionError, TooManyRedirects
import unicodedata
import mysql.connector

In [2]:
con = mysql.connector.connect(user='root', password='', host='127.0.0.1', database='news_db')

### Handling Baca Juga

In [ ]:
raw = pd.read_sql("select id, url, source from article where source='mobil123' and content like '%temukan mobil idaman dimobil123%' and content like '%mari bergabung bersama kami difacebookdantwitter%'", con)
for index, row in raw.iterrows():
    print(row['url'])
    response = requests.get(row['url'])
    html = response.text
    soup = BeautifulSoup(html, "html5lib")
    
    article = soup.find('div', class_="article__story-more soft--sides push--sides palm-hard palm-flush")
    
    for baca in article.findAll('p'):
        if "baca juga" in baca.get_text(strip=True).lower():
            baca.decompose()

    for link in article.findAll('div'):
        link.decompose()

    for link in article.findAll('small'):
        link.decompose() 

    for temu in article.findAll('p'):
        if "temukan mobil idaman dimobil123" in temu.get_text(strip=True).lower():
            temu.decompose()

    for mari in article.findAll('p'):
        if "mari bergabung bersama kami difacebookdantwitter" in mari.get_text(strip=True).lower():
            mari.decompose()
        
    detail = BeautifulSoup(article.decode_contents().replace('<br/>', ' '), "html5lib")
    content = re.sub(r'\n|\t|\b|\r','',unicodedata.normalize("NFKD",detail.get_text(strip=True)))
    content = content.replace("‘","").replace("‘","").replace("'","").replace(""" 
    "
    ""","")
                
    cursor = con.cursor()
    sql = "UPDATE article SET content = '"+content+"' WHERE id = "+str(row['id'])
    cursor.execute(sql)
    con.commit()
    cursor.close()